In [53]:
import random
from deap import base, creator, tools

class Train:
    def __init__(self, wagons, op, licence_plate):
        self.wagons = wagons
        self.op = op
        self.licence_plate = licence_plate

    def __str__(self):
        return "Número de vagones:" + str(self.wagons) \
        + "\n" + "Muelle de operaciones:" + str(self.op) \
        + "\n" + "Matrícula:" + str(self.licence_plate)


def random_trains_generation(n):

    train_list = []
    
    for i in range(n):
        wagons = random.randint(10, 30) 
        op = random.choice(["op1", "op2", "op3"])  
        train_list.append(Train(wagons, op, i))

    return train_list
    


incoming_trains = random_trains_generation(100)

def create_queue():
    aux = incoming_trains[:]
    random.shuffle(aux)
    return aux

class Trains(list):
    def __init__(self):
        list.extend(self, create_queue())

def evaluate(individual):
    ops = {"op1": 0, "op2": 0, "op3": 0}
    tiempo_total = 0

    for train in individual:
        if train.op in ops:
            if ops[train.op] == 0:
                ops[train.op] = train.wagons
            else:
                tiempo_total += ops[train.op]
                for op in ops:
                    if op != train.op:
                        ops[op] -= ops[train.op]
                        if ops[op] < 0:
                            ops[op] = 0
                ops[train.op] = train.wagons

    tiempo_total += max(ops.values())

    return tiempo_total,



In [54]:
import numpy as np
def train_mate(i1, i2):
    index = np.random.randint(0, len(i1) - 1)
    trains = set(i1)
    left1, right1 = i1[:index], i1[index:]
    left2, right2 = i2[:index], i2[index:]
    faltantes1 = trains - set(left1) - set(right2)
    faltantes2 = trains - set(left2) - set(right1)
    new_rigth1 = []
    for train in right2:
        if train not in left1:
            new_rigth1.append(train)
        else:
            new_rigth1.append(faltantes1.pop(0))
    new_rigth2 = []
    for train in right1:
        if train not in left2:
            new_rigth2.append(train)
        else:
            new_rigth2.append(faltantes2.pop(0))
    i1[:] = left1 + new_rigth2
    i2[:] = left2 + new_rigth1
    return i1, i2
    

In [55]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", Trains, fitness=creator.FitnessMin)
toolbox = base.Toolbox()
toolbox.register("individual", creator.Individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", evaluate)
toolbox.register("mate", train_mate)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.03)
toolbox.register("select", tools.selTournament, tournsize=3)


pop = toolbox.population(n=50)

In [57]:
from deap import algorithms
hof = tools.HallOfFame(1)

stats = tools.Statistics(lambda indiv: indiv.fitness.values)
stats.register("avg", np.mean)
stats.register("min", np.min)
stats.register("max", np.max)

pop, logbook = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=300, stats=stats, halloffame=hof, verbose=True)
print("Best individual is: %s\n with fitness: %s" % (hof[0], hof[0].fitness))
best_order = []
for train in hof[0]:
    print(train.licence_plate, train.op, train.wagons)
    best_order.append(train.licence_plate)
print("Best order of trains: ", best_order)
print("Best time: ", hof[0].fitness.values[0])
print(sorted(best_order))
print(len(best_order))

gen	nevals	avg  	min	max
0  	0     	889.8	870	970
1  	35    	885.72	870	943
2  	27    	880.92	870	946
3  	37    	879.48	870	947
4  	25    	878.24	870	955
5  	37    	875   	870	954
6  	26    	871.84	864	922
7  	24    	876.56	864	950
8  	28    	876.54	864	984
9  	24    	875.04	864	997
10 	29    	871.72	858	967
11 	30    	874.18	858	973
12 	32    	869.86	855	927
13 	29    	869.38	855	924
14 	30    	866.06	855	965
15 	25    	862.86	855	932
16 	21    	862.92	841	955
17 	32    	862.34	841	941
18 	33    	857.96	838	906
19 	32    	861.86	838	975
20 	26    	857.1 	838	972
21 	32    	853.2 	837	912
22 	33    	847.76	829	926
23 	32    	847.66	832	952
24 	29    	843.42	832	907
25 	29    	842.16	825	904
26 	33    	842.74	825	937
27 	32    	839.36	825	935
28 	35    	838.74	825	943
29 	26    	834.36	825	904
30 	27    	832.48	825	880
31 	28    	833.86	825	954
32 	27    	830.86	825	900
33 	30    	834.1 	815	911
34 	29    	832.2 	815	913
35 	29    	832.94	815	939
36 	39    	838.54	815	940
37 	27    	828